In [1]:
import os
os.chdir("..")
print("Current Directory:", os.getcwd())

Current Directory: /workspace/iscat


In [2]:
from src.data_processing.dataset import iScatDataset
from src.data_processing.utils import Utils
import torch
DEVICE= 'cuda:7' if torch.cuda.is_available() else 'cpu'
data_path_1 = os.path.join('dataset', '2024_11_11', 'Metasurface', 'Chip_02')
data_path_2 = os.path.join('dataset', '2024_11_12', 'Metasurface', 'Chip_01')
image_paths= []
target_paths=[]
image_indicies = 6
for data_path in [data_path_1,data_path_2]:
    i,t = Utils.get_data_paths(data_path,'Brightfield',image_indicies )
    image_paths.extend(i)
    target_paths.extend(t)

In [3]:
image_size=256
fluo_masks_indices=[1]
seg_method = "comdet"
normalize=False
train_dataset = iScatDataset(image_paths[:-2], target_paths[:-2], preload_image=True,image_size = (image_size,image_size),apply_augmentation=True,normalize=normalize,device=DEVICE,fluo_masks_indices=fluo_masks_indices,seg_method=seg_method)
valid_dataset = iScatDataset(image_paths[-2:],target_paths[-2:],preload_image=True,image_size = (image_size,image_size),apply_augmentation=False,normalize=normalize,device=DEVICE,fluo_masks_indices=fluo_masks_indices,seg_method=seg_method)

Loading TIFF images to Memory: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 18.12it/s]


In [4]:
MEAN = train_dataset.images.mean(dim=(0,2,3),keepdim=True)
STD = train_dataset.images.std(dim=(0,2,3),keepdim=True)

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from monai.losses import DiceLoss
from monai.metrics import MeanIoU
import numpy as np
from monai.networks.utils import one_hot
import random
def set_random_seed(seed):
    """
    Set the random seed for reproducibility in Python, NumPy, PyTorch, and CUDA.

    Args:
        seed (int): The seed value to use.
    """
    random.seed(seed)  # Python random
    np.random.seed(seed)  # NumPy
    torch.manual_seed(seed)  # PyTorch CPU
    torch.cuda.manual_seed(seed)  # PyTorch GPU
    torch.cuda.manual_seed_all(seed)  # PyTorch all GPUs
    
    # Ensure deterministic behavior (may slow down performance)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def z_score_normalize(images,mean=MEAN,std=STD, eps: float = 1e-8):
    normalized_images = (images - mean) / (std + eps)
    return normalized_images
    
miou_metric = MeanIoU(include_background=True, reduction="mean")
def compute_metrics(predictions: torch.Tensor, targets: torch.Tensor):
    """
    Compute mean IoU metric
    predictions: (batch_size, 1, height, width)
    targets: (batch_size, height, width)
    """

    pred_masks = (torch.softmax(predictions,dim=1) > 0.5).float()

    # Add channel dimension to targets if needed
    if len(targets.shape) == 3:
        targets = targets.unsqueeze(1)
    pred_one_hot=pred_masks
    target_one_hot = torch.cat([1 - targets, targets], dim=1) 
    # Compute IoU
    metric = miou_metric(pred_one_hot, target_one_hot)
    
    # Return mean IoU (average across classes)
    return metric.nanmean().item() 
        
class MultiClassUNet(nn.Module):
    def __init__(self, in_channels=3, num_classes=1, init_features=32):
        super(MultiClassUNet, self).__init__()
        
        # Load the pretrained model and modify the final layer
        model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet', 
                               in_channels=in_channels, 
                               out_channels=1, 
                               init_features=init_features, 
                               pretrained=False)
        
        # Replace the final convolution layer to match number of classes
        model.conv = nn.Conv2d(init_features, num_classes, kernel_size=1)
        
        self.model = model
    
    def forward(self, x):
        return self.model(x)

In [6]:
model_path_0_3 = "experiments/runs/UNet_Metasurface_2025-01-10_01-01-59/best_model.pth"
model_path_0_6 = "experiments/runs/UNet_Metasurface_2025-01-10_15-15-58/best_model.pth"
model_path_1_6 = "experiments/runs/UNet_Metasurface_2025-01-10_15-46-48/best_model.pth"
model_2 = MultiClassUNet(in_channels=6, num_classes=2, init_features=64)
checkpoint = torch.load(model_path_1_6, weights_only=False)
model_2.load_state_dict(checkpoint['model_state_dict'])
model_2.to(DEVICE)
model_2.eval()

Using cache found in /root/.cache/torch/hub/mateuszbuda_brain-segmentation-pytorch_master


MultiClassUNet(
  (model): UNet(
    (encoder1): Sequential(
      (enc1conv1): Conv2d(6, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (enc1norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (enc1relu1): ReLU(inplace=True)
      (enc1conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (enc1norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (enc1relu2): ReLU(inplace=True)
    )
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (encoder2): Sequential(
      (enc2conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (enc2norm1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (enc2relu1): ReLU(inplace=True)
      (enc2conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (enc2norm

In [7]:
import numpy as np
import torch

def channel_ablation(model, inputs, targets, channel_idx, device):
    input_ = inputs.clone().to(device)
    input_ = z_score_normalize(inputs)
    ablated_inputs = input_.clone()
    ablated_inputs[:, channel_idx, :, :] = 0 
    print((ablated_inputs==input_).all())# Set one channel to zero
    with torch.no_grad():
        abla_outputs = model(ablated_inputs)
        outputs = model(input_)
    abla_miou = compute_metrics(abla_outputs, targets.clone().to(device))
    miou = compute_metrics(outputs, targets.clone().to(device))# Define your loss metric
    out = round(abla_miou*100/miou)
    return out

In [14]:
batch_size=150
from torch.utils.data import DataLoader, Dataset

def create_dataloaders(train_dataset, test_dataset, batch_size=4):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader
train_loader, val_loader = create_dataloaders(train_dataset, valid_dataset, batch_size=batch_size)

In [15]:
input_samples,target_samples = next(iter(val_loader)) 

In [16]:
mious = []
for i in range(input_samples.shape[1]):  # Loop over channels
    miou = channel_ablation(model_2, input_samples, target_samples, i, DEVICE)
    mious.append(miou)
print("miou per channel:", mious)

tensor(False, device='cuda:7')
tensor(False, device='cuda:7')
tensor(False, device='cuda:7')
tensor(False, device='cuda:7')
tensor(False, device='cuda:7')
tensor(False, device='cuda:7')
miou per channel: [91, 91, 85, 91, 89, 95]


In [11]:
print("miou per channel:", mious)

miou per channel: [95, 97, 90, 92, 94, 101]
